In [1]:
from __future__ import division
import cv2 
import numpy as np
import glob
import csv
import os
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import joblib

In [2]:
import glob
import os
import numpy as np
import cv2

# Path folder utama yang berisi subfolder untuk setiap kelas
folder_utama = './dataset/train'

# Membaca semua subfolder di dalam folder utama
kelas_folders = glob.glob(os.path.join(folder_utama, '*'))

for kelas_folder in kelas_folders:
    # Membaca semua file gambar dalam subfolder kelas
    path = os.path.join(kelas_folder, '*.*')
    
    for bb, file in enumerate(glob.glob(path)):
        # Membuat kernel untuk nanti masking
        kernelOpen = np.ones((5, 5))
        kernelClose = np.ones((20, 20))

        # Baca gambar rgb
        rgb = cv2.imread(file)

        # Convert BGR to HSV
        hsv = cv2.cvtColor(rgb, cv2.COLOR_BGR2HSV)

        lower_white = np.array([0, 0, 168])
        upper_white = np.array([172, 111, 255])

        whitemask = cv2.inRange(hsv, lower_white, upper_white)

        # Invers mask putih agar yang terdeteksi adalah yang bukan putih
        non_whitemask = cv2.bitwise_not(whitemask)

        # Add a range for grey colors (adjust the trainues based on your needs)
        lower_grey = np.array([0, 0, 100])
        upper_grey = np.array([172, 30, 255])

        # Create a mask for grey colors
        greymask = cv2.inRange(hsv, lower_grey, upper_grey)

        # Combine masks to exclude white and grey regions
        non_whitemask_no_grey = cv2.bitwise_and(non_whitemask, cv2.bitwise_not(greymask))

        # Gunakan operasi open dengan kernel yang tadi di atas
        maskOpen = cv2.morphologyEx(non_whitemask_no_grey, cv2.MORPH_OPEN, kernelOpen)

        # Hasil dari open dipakai di close ini
        maskClose = cv2.morphologyEx(maskOpen, cv2.MORPH_CLOSE, kernelClose)

        # Hasil akhirnya masking dari close
        maskFinal = maskClose

        # Setelah itu gunakan bitwise untuk menyatukan gambarnya, hanya menggunakan hsv bukan rgbnya
        maskFinally = cv2.bitwise_and(hsv, hsv, mask=maskFinal)

        # Convert back to RGB
        masked_rgb = cv2.cvtColor(maskFinally, cv2.COLOR_HSV2BGR)

        # Resize gambar ke 256x256
        resized_image = cv2.resize(masked_rgb, (256, 256), interpolation=cv2.INTER_AREA)

        # Simpan gambar hasil masking dan resizing sesuai dengan struktur folder
        output_folder = os.path.join('train', os.path.basename(kelas_folder))
        os.makedirs(output_folder, exist_ok=True)
        
        # Gunakan label kelas sebagai nama file
        class_label = os.path.basename(kelas_folder)
        output_filename = f'masking_{class_label}_{bb}.jpg'
        output_path = os.path.join(output_folder, output_filename)
        
        try:
            cv2.imwrite(output_path, maskFinally)
            print(f"Saved: {output_filename}")
        except Exception as e:
            print(f"Error saving {output_filename}: {e}")

print("All images have been processed and saved.")


Saved: masking_Manggis_0.jpg
Saved: masking_Manggis_1.jpg
Saved: masking_Manggis_2.jpg
Saved: masking_Manggis_3.jpg
Saved: masking_Manggis_4.jpg
Saved: masking_Manggis_5.jpg
Saved: masking_Manggis_6.jpg
Saved: masking_Manggis_7.jpg
Saved: masking_Manggis_8.jpg
Saved: masking_Manggis_9.jpg
Saved: masking_Manggis_10.jpg
Saved: masking_Manggis_11.jpg
Saved: masking_Manggis_12.jpg
Saved: masking_Manggis_13.jpg
Saved: masking_Manggis_14.jpg
Saved: masking_Manggis_15.jpg
Saved: masking_Manggis_16.jpg
Saved: masking_Manggis_17.jpg
Saved: masking_Manggis_18.jpg
Saved: masking_Manggis_19.jpg
Saved: masking_Manggis_20.jpg
Saved: masking_Manggis_21.jpg
Saved: masking_Manggis_22.jpg
Saved: masking_Manggis_23.jpg
Saved: masking_Manggis_24.jpg
Saved: masking_Manggis_25.jpg
Saved: masking_Manggis_26.jpg
Saved: masking_Manggis_27.jpg
Saved: masking_Manggis_28.jpg
Saved: masking_Manggis_29.jpg
Saved: masking_Manggis_30.jpg
Saved: masking_Manggis_31.jpg
Saved: masking_Manggis_32.jpg
Saved: masking_Mangg